- First start with Jane Austen books tokenized. 
- Using original word at this point.
- N-gram window needs to through one column across n-rows.

In [1]:
import os, sys, re, json, time, csv, copy, random, unittest
import itertools, collections

import numpy as np
from scipy import stats

import nltk

from shared_lib import utils, vocabulary, ngram_lm, ngram_utils

random.seed(10)

In [2]:
austen_emma_all = []
with open('../book-nlp-master/data/tokens.gutenberg/austen.pride.tokens', 'rb') as f:
    reader = csv.reader(f, dialect="excel-tab")
    austen_emma_all = list(reader)
# Remove header
austen_emma_all = austen_emma_all[1:]

In [60]:
import pandas as pd

df = pd.read_csv('../book-nlp-master/data/tokens.gutenberg/Henry_James___The_Ambassadors.tokens', sep='\t', quoting=csv.QUOTE_NONE)

df.query('characterId > -1 & ner == "PERSON" & inQuotation == False')['sentenceID']['characterId']
# characters = {}
# for row in df.query('characterId > -1 & ner == "PERSON" & inQuotation == False'):
#     print type(row)
#     characters['id'] = row['characterId']
#     if characters['id']['dialogs'] is None:
#         characters['id']['dialogs'] = []
#     characters['id']['dialogs'].append(' '.join(df.query('sentenceID == ' + row['sentenceID'])['originalWord']))
# characters
    


KeyError: 'characterId'

In [40]:
' '.join(df.query('sentenceID == 86')['originalWord'])
' '.join(df.query('sentenceID == 5748')['originalWord'])
' '.join(df.query('sentenceID == 99')['originalWord'])
' '.join(df.query('sentenceID == 9116')['originalWord'])

'And what IS beastly , at all events , " he added , " is losing you . "'

In [3]:
austen_emma_tokens = [line[7] for line in austen_emma_all]
print 'The first 10 tokens from Jane Austen\'s Emma:'
print ', '.join(austen_emma_tokens[0:30])

The first 10 tokens from Jane Austen's Emma:
Produced, by, An, Anonymous, Volunteer, EMMA, By, Jane, Austen, VOLUME, I, CHAPTER, I, Emma, Woodhouse, ,, handsome, ,, clever, ,, and, rich, ,, with, a, comfortable, home, and, happy, disposition


In [4]:
# Shuffle when needed
random.shuffle(austen_emma_tokens)

In [4]:
tokens_length = len(austen_emma_tokens)
print tokens_length * 0.8
V = 40000
train_tokens  = austen_emma_tokens[ : int(tokens_length * 0.8)]
test_tokens = austen_emma_tokens[int(tokens_length * 0.2) : ]
vocab = vocabulary.Vocabulary((utils.canonicalize_word(w) for w in train_tokens), size=V)
print "Train set vocabulary: %d words" % vocab.size
print "Train set tokens: %d " % len(train_tokens)
print "Test set tokens: %d " % len(test_tokens)
print "First 10 Train Tokens: ", train_tokens[0:20]
print "First 10 Test Tokens: ", test_tokens[0:20]

154540.0
Train set vocabulary: 6689 words
Train set tokens: 154540 
Test set tokens: 154540 
First 10 Train Tokens:  ['Produced', 'by', 'An', 'Anonymous', 'Volunteer', 'EMMA', 'By', 'Jane', 'Austen', 'VOLUME', 'I', 'CHAPTER', 'I', 'Emma', 'Woodhouse', ',', 'handsome', ',', 'clever', ',']
First 10 Test Tokens:  ['were', 'thus', 'comfortably', 'occupied', ',', 'Mr.', 'Woodhouse', 'was', 'enjoying', 'a', 'full', 'flow', 'of', 'happy', 'regrets', 'and', 'fearful', 'affection', 'with', 'his']


In [5]:
Model = ngram_lm.KNTrigramLM
t0 = time.time()
print "Building trigram LM...",
lm = Model(train_tokens)
print "done in %.02f s" % (time.time() - t0)
ngram_utils.print_stats(lm)

Building trigram LM... done in 0.87 s
=== N-gram Language Model stats ===
7080 unique 1-grams
54967 unique 2-grams
113902 unique 3-grams
Optimal memory usage (counts only): 3 MB


In [7]:
max_length = 20
num_sentences = 5

for _ in range(num_sentences):
    seq = ["<s>", "<s>"]
    for i in range(max_length):
        seq.append(ngram_utils.predict_next(lm, seq))
        # Stop at end-of-sentence.
        if seq[-1] == "</s>": break
    print " ".join(seq)
    print "[{1:d} tokens; log P(seq): {0:.02f}]".format(*ngram_utils.score_seq(lm, seq))
    print ""

<s> <s> Highbury world ; “ circumstances absence John but thinking I my , not a only but days 10,000 am deal
[20 tokens; log P(seq): -183.12]

<s> <s> obliging own any sorry every ’s ” as but it -- . nothing a it had ” us allow _
[20 tokens; log P(seq): -139.10]

<s> <s> is much to state like carriage well dear of . was understand , rendered prevent , they as must any
[20 tokens; log P(seq): -145.86]

<s> <s> any ? the a match the had he , of and who find make in Emma as He very been
[20 tokens; log P(seq): -149.63]

<s> <s> little better and himself London I Mr. came Elton ought am was so indulge , be and a ordinarily risking
[20 tokens; log P(seq): -155.35]



In [8]:
lm.set_live_params(k = 0.001, delta=0.75)

In [9]:
log_p_data, num_real_tokens = ngram_utils.score_seq(lm, train_tokens)
print "Train perplexity: %.02f" % (2**(-1*log_p_data/num_real_tokens))

Train perplexity: 11.01


In [10]:
len(test_tokens)

154540

In [12]:
log_p_data, num_real_tokens = ngram_utils.score_seq(lm, test_tokens)
print "Test perplexity: %.02f" % (2**(-1*log_p_data/num_real_tokens))

Test perplexity: 29.13
